In [ ]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import matplotlib.pyplot as plt
from tqdm import tqdm

import time , os ,sys

In [ ]:
# to view the tweet text 
pd.set_option('display.max_colwidth', None)

# table of contents :
   - [data gathering](#data-gathering)
   - [assising result](#assising-result)
   - [analysis](#analysis)
     

# data gathering

* read from `twitter-archive-enhanced.csv`
* download `image_predictions.tsv`

* use tweepy to load
    - retweet count
    - like count
    - ...
* save tweater data to `tweet_json.txt` and load them as df

#### read from `twitter-archive-enhanced.csv`

In [ ]:
twitter_acrchive = pd.read_csv('twitter-archive-enhanced.csv')

#### download `image_predictions.tsv`

In [ ]:
# download the file if not already downloades
if not os.path.exists('image_predictions.tsv'):
    response = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')
    with open('image_predictions.tsv','wb') as file:
        file.write(response.content)    

In [ ]:
image_predictions = pd.read_csv('image_predictions.tsv',sep='\t')

**use tweepy to load twtweets data**
    - retweet count
    - like count
    - ...


In [ ]:
consumer_key = 'JJW6Y1yUkRzWORLJp3AsnoNS0'
consumer_secret = '7HOeyP0Q8f749qFE2E6hIACoFwgHYCv2nOIbLpblcIqJStZb7o'
access_token = '3084174784-vfOMmfQcZTFl3m5loSs5BQfYcfxHg3epEKcUAxH'
access_secret = '5tEPXKzPcxHBRaZzM0mzb7euKG2vZrE5uOkbO95BSoNfD'


auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

## http://docs.tweepy.org/en/v3.2.0/api.html#API

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

- https://tweepy.readthedocs.io/en/latest/api.html#API.get_status
- http://docs.tweepy.org/en/latest/extended_tweets.html?highlight=tweet_mode#extended-mode
- http://docs.tweepy.org/en/latest/extended_tweets.html?highlight=tweet_mode#examples

In [ ]:
failed_ids = []


if not os.path.exists('tweet_json.txt'):
    with open('tweet_json.txt','w') as tweets_file:
        for tweet_id in tqdm(twitter_acrchive.tweet_id):
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                json.dump(tweet._json, tweets_file)
                tweets_file.write('\n')
            except Exception as e:
                failed_ids.append(tweet_id)
    # summery 
    num_ids = len(twitter_acrchive.tweet_id)
    num_failed = len(failed_ids)
    num_success = num_ids - num_failed

    print('--------------------------\n')
    print('sucessfull = ' + str(num_success))
    print('failed = ' + str(num_failed))
    print('------ failed tweets ---------------------')
    print(failed_ids)
else:
    print('file already exist please remove it if not complate')

* save tweater data to `tweet-json.txt` 
* convert data to dict then to df

In [ ]:
rows = []
with open('tweet_json.txt','r') as json_file:
    for line in json_file.readlines():
        data = json.loads(line)
        data['favorite_count']
        
        rows.append({'tweet_id': data['id'],
                     'retweet_count':  data['retweet_count'],
                     'favorite_count': data['favorite_count']})

In [ ]:
twitter_extra = pd.DataFrame(rows)

### save loaded data to csv

In [ ]:
# save original data
image_predictions.to_csv('image_predictions.csv', index=False)
twitter_extra.to_csv('twitter_extra.csv', index=False)

# Assessing Data

### check any error in rating , dog stages

quate
```
I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages .
```




#### [assess_1](#assising-result)

In [ ]:
calculated_score =  twitter_acrchive.rating_numerator.astype(str) + '/' + twitter_acrchive.rating_denominator.astype(str)

In [ ]:
tweete_score = twitter_acrchive.text.str.extract(r'(\d+\/\d+)')[0]

In [ ]:
twitter_acrchive.loc[calculated_score!=tweete_score,['tweet_id','text','rating_numerator','rating_denominator']]

* `835246439529840640` has two rating like text

* `826598799820865537` is  comment , the rated was in other post

https://twitter.com/dog_rates/status/826598799820865537

#### [asses_2](#assising-result)

In [ ]:
twitter_acrchive[['rating_numerator','rating_denominator']].describe()

In [ ]:
strange_numerator = twitter_acrchive[(twitter_acrchive.rating_numerator < 10) | (twitter_acrchive.rating_numerator > 20)]
strange_numerator[['tweet_id','text','rating_numerator','rating_denominator']]

* rating with fraction doesn't get the whole number

#### [asses_3](#assising-result)

In [ ]:
strange_denominator = twitter_acrchive[twitter_acrchive.rating_denominator != 10]
strange_denominator[['tweet_id','text','rating_numerator','rating_denominator']]

* `810984652412424192` has no rating 
* `666287406224695296`,`835246439529840640` ..  has two rating like text
* `758467244762497024` denomrator is not 10 

### check other column

#### [asses_4](#assising-result)


In [ ]:
twitter_acrchive.head()

- complex values[url, string] in source coloumn 
- null values in [retweeted_status_id	retweeted_status_user_id	retweeted_status_timestamp]
, [in_reply_to_status_id	in_reply_to_user_id]

#### [asses_5](#assising-result)

In [ ]:
twitter_acrchive.info()

- wrong dtypes: `[timestamp,retweeted_status_timestamp]`
- dog type in represented in multiple columns (doggo  floofer pupper puppo)
- column source is not descriptive


#### [asses_6](#assising-result)


In [ ]:
image_predictions.info()

- image prediction multible coulmns for beerd 
- number of tweats is higher than number of images
-------------------------------------

#### [asses_7](#assising-result)

In [ ]:
image_predictions[~image_predictions.p1_dog & ~image_predictions.p2_dog & ~image_predictions.p3_dog]

* some images has no dogs

#### [asses_8](#assising-result)
* programitically extract score for multible image tweats is requires manual image inspection

#### [asses_9](#assising-result)

In [ ]:
twitter_extra.info()

- tweet extra has related data to tweater_archive 

# assising result

[analysis](#analysis)

* note: asses anchor is written after the observation
## ROUND 1
### messy
- [x] tweet extra has related data to tweater_archive \[[assessing](#asses_9)\] \[[tide](#tide_1)\]
- [x] dog type in represented in column names (doggo  floofer pupper puppo)  \[[assessing](#asses_5)\] \[[tide](#tide_2)\]
- [x] name column `source`  is not descriptive \[[assessing](#asses_5)\] \[[tide](#tide_4)\]
- [x] image prediction multible coulmns for beard `hard analysis` \[[assessing](#asses_6)\] \[[tide](#tide_3)\]
- [x] dog breed can be added to Twitter archive  \[[tide](#tide_5)\]


### dirty
* tweater_archive
    - [x] complex values in source coloumn \[[assessing](#asses_4)\] \[[clean](#clean_1)\]
    - [x] rating with fraction takes fraction only _ex.. `9.5/10` is saved as `5/10`
    \[[assessing](#asses_2)\] \[[clean](#clean_2)\]
        - [x] `181` tweats that have retweet_status and `78` replays \[[clean](#clean_4)\]
    ```
    You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
    ```
    - [x] stages[doggo , floofer ..] and name columns use string `None` or `none` to represent null value [solved in tidy stage] 
    - [x] wrong dtypes  \[[assessing](#asses_5)\] \[[clean](#clean_6)\]
        * tweater_archive: timestamp, *after data tidtidying* [bread,type]

* image preditictions
    - [x] some images has no dogs \[[assessing](#asses_7)\] \[[clean](#clean_5)\]
    - [x] number of tweets is higher than number of predicted images \[[assessing](#asses_6)\] \[[clean](#clean_7)\]
    - [x] dog bread is catagorial dtype \[[clean](#clean_6)\]


# ROUND 2 _after cleaning
### dirty
- [x] some dogs has multipe dog_stage \[[assessing](#asses_2_1)\] \[[clean](#clean_8)\]
    

# cleanning

## make copy

In [ ]:
twitter_acrchive_clean =  twitter_acrchive.copy()
twitter_extra_clean = twitter_extra.copy()
image_predictions_clean = image_predictions.copy()

## tidness issues

#### [tide_1](#assising-result)

### tweet extra has related data to twitter_archive 


solution:
* merge the two data frames with tweet_id

In [ ]:
twitter_acrchive_clean = twitter_acrchive_clean.merge(twitter_extra, on='tweet_id')


## test

In [ ]:
twitter_acrchive_clean.columns

#### [tide_2](#assising-result)
### dog type is represented in column name (doggo  floofer pupper puppo)



solution:
- create column named `type` that holds this value 
- drop those columns 

In [ ]:
# extract other columns names
id_list = list(twitter_acrchive_clean.columns)
dog_types = ['doggo','floofer','pupper','puppo']
id_list = [x for x in id_list if x not in dog_types]

# melt types columns
dog_type = twitter_acrchive_clean.melt(id_vars=id_list,value_name='dog_type')
dog_type = dog_type[['tweet_id','dog_type']]

In [ ]:
# remove the records with None and add the new column 
dog_type = dog_type[dog_type.dog_type != 'None']
twitter_acrchive_clean = twitter_acrchive_clean.merge(dog_type,on='tweet_id',how='left')

In [ ]:
twitter_acrchive_clean.drop(columns=dog_types, inplace=True)

# test

In [ ]:
twitter_acrchive_clean.info()

#### asses_2_1
-------------------------------------
# dirty data
* the count increased from `2356` to `2368` 
    - some dogs has multipe types

#### [tide_3](#assising-result)

### image prediction represent  beard in multible coulmns

* take the most confidant dog bread
    1. change false in `p1,p2,p3` coulmns with null
    2. use `combine` to get columns with non null value betwean `p1,p2`
    3. reapeat 2 for `last_result,p3`
    4. drop useless coulmns

In [ ]:
# replace false breed with null
image_predictions_clean.loc[~image_predictions_clean.p1_dog,['p1']] = np.nan
image_predictions_clean.loc[~image_predictions_clean.p1_dog,['p1_conf']] = np.nan


image_predictions_clean.loc[~image_predictions_clean.p2_dog,['p2']] = np.nan
image_predictions_clean.loc[~image_predictions_clean.p2_dog,['p2_conf']] = np.nan

image_predictions_clean.loc[~image_predictions_clean.p3_dog,['p3']] = np.nan
image_predictions_clean.loc[~image_predictions_clean.p3_dog,['p3_conf']] = np.nan

In [ ]:
# choose non false breed p1,p2
image_predictions_clean['breed'] = image_predictions_clean.p1.combine_first(image_predictions_clean.p2)
image_predictions_clean['breed_confidance'] = image_predictions_clean.p1_conf.combine_first(image_predictions_clean.p2_conf)

In [ ]:
# choose non false breed last_result,p3
image_predictions_clean['breed'] = image_predictions_clean.breed.combine_first(image_predictions_clean.p3)
image_predictions_clean['breed_confidance'] = image_predictions_clean.breed_confidance.combine_first(image_predictions_clean.p3_conf)

* naming was missy

In [ ]:
image_predictions_clean.drop(columns=['p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
                       inplace=True)

## test

In [ ]:
image_predictions_clean.info()

#### [tide_4](#assising-result)


## column source is not descriptive
- rename to `source_device`

In [ ]:
twitter_acrchive_clean.rename(columns={'source':'source_device'}, inplace=True)

In [ ]:
twitter_acrchive_clean.info()

# ditry data


#### [clean_1](#assising-result)

###  complex values in source coloumn 
- select text in <h> tag body 
- convert to simpler values

In [ ]:
tweete_source = twitter_acrchive_clean.source_device.str.extract(r'(>.*<)')[0]

In [ ]:
twitter_acrchive_clean.source_device = tweete_source.str[1:-1]


In [ ]:
map_simple_source = {
    'Twitter for iPhone' : 'iPhone',
    'Twitter Web Client' : 'Web Client',
    'Vine - Make a Scene' : 'Vine',
    'TweetDeck' : 'TweetDeck'
}
twitter_acrchive_clean.source_device.replace(map_simple_source, inplace=True)

## test 

In [ ]:
twitter_acrchive_clean.source_device.value_counts()

### rating problems

#### [clean_2](#assising-result)

#### rating with fraction takes fraction only _ex.. `9.5/10` is saved as `5/10`_


### sol:
* use regex expration that take the point into acount

observed cases:
    - 9.5/10 
    - ...10/10 
    - 13/10

In [ ]:
tweete_score = twitter_acrchive_clean.text.str.extract(r'(\d+\.?\d*\/\d+\.?\d*)')[0]

In [ ]:
twitter_acrchive_clean[['rating_numerator','rating_denominator']] =  tweete_score.str.split('/',expand=True).astype(float)

## test

In [ ]:
twitter_acrchive_clean.loc[twitter_acrchive_clean.rating_numerator%1!=0,
                           ['text','rating_numerator','rating_denominator']]

#### [clean_3](#assising-result)

#### tweets with rating like value 

## define

#### double rating cases
##### real doubles or 10/10 => take the first one only
- two dogs two ratings 
- one dog two 10/10

##### fake double => take last rating
- 24/7
- joking then rate
- contain date 4/20

## sol

#### sol [real double]:
- select tweats with `and` or `&` or `10/10`
- choose the first rate as the right rating



#### sol [fake double]:
- select tweats without `and` or `&` or `10/10`
- choose the last rate as the right rating



In [ ]:
match_all = twitter_acrchive_clean.text.str.extractall(r'(\d+\.?\d*\/\d+\.?\d*)')
double_rated = twitter_acrchive_clean.iloc[match_all.xs(1,level=1).index]
double_rated = double_rated[['tweet_id','name','text','rating_numerator','rating_denominator']]

In [ ]:
fake_double = double_rated[(double_rated.text.str.find('and') < 0) &
                            (double_rated.text.str.find('&') < 0) &
                            (double_rated.text.str.find('10/10') < 0)]

real_double = double_rated.drop(fake_double.index)

In [ ]:
# take the second rate for fake doubles
second_valid = match_all.xs(1,level=1).loc[fake_double.index][0]
second_index = second_valid.index
second_nom_dem = second_valid.str.split('/',expand=True).astype(float)

twitter_acrchive_clean.loc[second_index,['rating_numerator']] = second_nom_dem[0]
twitter_acrchive_clean.loc[second_index,['rating_denominator']] = second_nom_dem[1]

# for real double first tweet is already taken

# test

In [ ]:
#fake_doubles
twitter_acrchive_clean.loc[second_index,['tweet_id','text','rating_numerator','rating_denominator']]

In [ ]:
#real_doubles
twitter_acrchive_clean.loc[real_double.index, ['tweet_id','name','text','rating_numerator','rating_denominator']]

#### [clean_5](#assising-result)

### some images has no dogs 

remove tweeets with null value at dog breed (the AI model coudn't detect)

In [ ]:
image_predictions_clean = image_predictions_clean[image_predictions_clean.breed.notna()]

# test

In [ ]:
twitter_acrchive_clean.shape

#### [clean_4](#assising-result)

### `181` tweats that have retweet_status,  `78` replies

- remove retweats and replies
- remove there associated coulmns

In [ ]:
retweet_index = twitter_acrchive_clean[twitter_acrchive_clean.retweeted_status_id.notnull()].index
twitter_acrchive_clean.drop(labels=retweet_index,inplace=True)

replay_index = twitter_acrchive_clean[twitter_acrchive_clean.in_reply_to_status_id.notnull()].index
twitter_acrchive_clean.drop(labels=replay_index,inplace=True)

In [ ]:
twitter_acrchive_clean.drop(columns=['in_reply_to_status_id','in_reply_to_user_id'],inplace=True)
twitter_acrchive_clean.drop(columns=['retweeted_status_id',
                                     'retweeted_status_user_id',
                                     'retweeted_status_timestamp'],inplace=True)

## test

In [ ]:
twitter_acrchive_clean.info()

# test

#### [clean_6](#assising-result)

## wrong dtypes
    * timedate: timestamp
    * catagorial : dog_type ,source_device

In [ ]:
twitter_acrchive_clean.timestamp = pd.to_datetime(twitter_acrchive_clean.timestamp)
twitter_acrchive_clean.dog_type = twitter_acrchive_clean.dog_type.astype('category')
twitter_acrchive_clean.source_device = twitter_acrchive_clean.source_device.astype('category')
image_predictions_clean.breed = image_predictions_clean.breed.astype('category')

## test

In [ ]:
twitter_acrchive_clean.info()

#### [clean_7](#assising-result)

### number of tweets is higher than number of predicted images

define : there are tweets that has no images or images without dogs we nee to exclude those from analysis

sol : 
- remove tweets whose id doesn't exist in image prediction data frame

In [ ]:
twitter_acrchive_clean = twitter_acrchive_clean[twitter_acrchive_clean.tweet_id.isin(image_predictions_clean.tweet_id)]
image_predictions_clean = image_predictions_clean[image_predictions_clean.tweet_id.isin(twitter_acrchive_clean.tweet_id)]

# test

In [ ]:
print(image_predictions_clean.shape)
print(twitter_acrchive_clean.shape)
# the difrance in size is result from duplicate in twitter_acrchive_clean

#### [clean_8](#assising-result)

### some dogs has multipe dog_stages

define : in tweet text we rate dogs may mention two dog stages which result in duplcate in data

sol : 
- remove duplicate tweets 


In [ ]:
twitter_acrchive_clean.drop_duplicates(subset=['tweet_id'],inplace=True)

# test

In [ ]:
# check if duplicate make them equal
print(image_predictions_clean.shape)
print(twitter_acrchive_clean.shape)

twitter_acrchive_clean.tweet_id.duplicated().sum()

# save data

In [ ]:
twitter_archive_master = twitter_acrchive_clean.merge(image_predictions_clean,on='tweet_id')

In [ ]:
twitter_archive_master.to_csv('twitter_archive_master.csv',index=False)
twitter_acrchive_clean.to_csv('twitter_acrchive_clean.csv',index=False)
image_predictions_clean.to_csv('image_predictions_clean.csv',index=False)

# analysis
- avrage rating for dog types, dogbreed
- most retweated/loved type
- is rating random ? 
- value count for types
- do people agree ? the likes compared to rating
- number of photos effect

In [ ]:
# master analysis 


In [ ]:
means_by_type = twitter_acrchive_clean.groupby('dog_type').mean()
means_by_type = means_by_type[['rating_numerator','retweet_count','favorite_count']]

In [ ]:
means_by_type

In [ ]:
means_by_type.plot.bar(subplots=True,figsize=(8,8),logy=True);

In [ ]:
twitter_acrchive_clean.rating_numerator[twitter_acrchive_clean.rating_numerator < 20].hist();

* seems scwed a bit the right but overall seems a normal distubtion not random values

## value count for types

In [ ]:
twitter_acrchive_clean.dog_type.value_counts()

In [ ]:
twitter_acrchive_clean.groupby(['source_device','dog_type']).count()['tweet_id']